In [2]:
import openai
import sys
sys.path.append("/gscratch/balazinska/enhaoz/complex_event_video")

from src.utils import str_to_program_postgres, postgres_execute_cache_sequence
from experiments.analysis.evaluate_gpt import evaluate_patsql
# Load your API key from an environment variable or secret management service
openai.api_key = open("/gscratch/balazinska/enhaoz/complex_event_video/experiments/gpt/openai_key.txt", "r").read()


## Near(o1, o2); Far(o1, o2)

In [4]:
# Experiment 1 
response = openai.Completion.create(
  model="code-davinci-002",
  prompt="""
/*    
All rows from table Near:
SELECT * FROM Near;
Table: Near 
oid1    oid2    vid    fid
0   1   2492    0
0   1   2492    1
0   1   2492    2
0   1   2492    3
0   1   2492    4
0   1   4752    4
0   1   5818    3
0   1   5818    4
*/ 
    
/*
All rows from table Far:    
SELECT * FROM Far;
Table: Far
oid1    oid2    vid    fid
0   1   595   0
0   1   595   1
0   1   595   2
0   1   595   3
0   1   595   4
0   1   595   5
0   1   595   6
0   1   595   7
0   1   595   31
0   1   836   0
0   1   836   1
0   1   836   2
0   1   836   3
0   1   836   4
0   1   836   5
0   1   836   6
0   1   836   7
0   1   836   8
0   1   836   9
0   1   836   10
0   1   836   11
0   1   836   12
0   1   836   13
0   1   836   14
0   1   836   15
0   1   836   16
0   1   836   17
0   1   836   18
0   1   836   19
0   1   836   20
0   1   836   21
0   1   836   22
0   1   836   23
0   1   836   24
0   1   836   25
0   1   836   26
0   1   836   27
0   1   1474   0
0   1   1474   1
0   1   1474   2
0   1   1474   3
0   1   1474   4
0   1   1474   5
0   1   1474   6
0   1   1474   7
0   1   1474   8
0   1   1474   9
0   1   1474   10
0   1   1474   11
0   1   1474   12
0   1   1474   13
0   1   1474   14
0   1   1474   15
0   1   1474   16
0   1   1474   17
0   1   1474   18
0   1   1474   19
0   1   1474   20
0   1   1474   21
0   1   1474   22
0   1   1474   23
0   1   1474   24
0   1   1474   25
0   1   1474   26
0   1   1474   27
0   1   2492   18
0   1   2492   19
0   1   2492   20
0   1   2492   21
0   1   2492   22
0   1   2492   23
0   1   2492   24
0   1   2492   25
0   1   2492   26
0   1   2492   27
0   1   2492   28
0   1   2492   29
0   1   2492   30
0   1   2492   31
0   1   2788   0
0   1   2788   1
0   1   2788   2
0   1   2788   25
0   1   2788   26
0   1   2788   27
0   1   2788   28
0   1   2788   29
0   1   2788   30
0   1   2788   31
0   1   2919   0
0   1   2919   1
0   1   2919   2
0   1   2919   3
0   1   2919   4
0   1   2919   5
0   1   2919   6
0   1   2919   7
0   1   2919   8
0   1   2919   9
0   1   2919   10
0   1   2919   11
0   1   2919   12
0   1   2919   13
0   1   2919   14
0   1   2919   15
0   1   2919   16
0   1   2919   17
0   1   2919   18
0   1   2919   19
0   1   2919   20
0   1   2919   21
0   1   2919   22
0   1   2919   23
0   1   2919   24
0   1   2919   25
0   1   2919   26
0   1   2919   27
0   1   2919   28
0   1   2919   29
0   1   2919   30
0   1   2919   31
0   1   4752   0
0   1   4752   1
0   1   4752   9
0   1   4752   13
0   1   4752   14
0   1   4752   15
0   1   4752   16
0   1   4752   17
0   1   6785   0
0   1   6785   1
0   1   6785   22
0   1   6785   23
0   1   6785   24
0   1   6785   25
0   1   6785   26
0   1   6785   27
0   1   6785   28
0   1   9887   0
0   1   9887   1
0   1   9887   2
0   1   9887   3
0   1   9887   4
0   1   9887   5
0   1   9887   6
0   1   9887   7
0   1   9887   8
0   1   9887   9
0   1   9887   10
0   1   9887   11
*/

--- Using valid Postgres SQL, but don't use subqueries, 
--- generate a query that, when executed on the tables provided above, returns the following table Output:

/*
All rows from table Output:    
SELECT * FROM Output;
Table: Output
vid
2492 
4752
*/
    
SELECT
  """,
  temperature=0,
  max_tokens=500,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["#", ";"]
)



output = response.choices[0].text.replace('\n', ' ')
output = ' '.join(output.split())
output 

'vid FROM Near WHERE fid IN ( SELECT fid FROM Far WHERE oid1 = 0 AND oid2 = 1 AND vid = 2492 ) AND oid1 = 0 AND oid2 = 1 AND vid != 2492 GROUP BY vid HAVING COUNT(*) = ( SELECT COUNT(*) FROM Far WHERE oid1 = 0 AND oid2 = 1 AND vid = 2492 )'

In [6]:
gt_query = "Near_1(o0, o1); Far_3(o0, o1)"
predicate_list = ("Near_1", "Far_3")
synthesized_query = 'SELECT vid FROM Near WHERE fid IN ( SELECT fid FROM Far WHERE oid1 = 0 AND oid2 = 1 AND vid = 2492 ) AND oid1 = 0 AND oid2 = 1 AND vid != 2492 GROUP BY vid HAVING COUNT(*) = ( SELECT COUNT(*) FROM Far WHERE oid1 = 0 AND oid2 = 1 AND vid = 2492 )'

evaluate_patsql(gt_query, predicate_list, 5432, 4, synthesized_query)

score: 0.0, runtime: 0.03313899040222168


In [11]:
# Experiment 2
# 20 examples
response = openai.Completion.create(
  model="code-davinci-002",
  prompt="""
/*    
All rows from table Near:
SELECT * FROM Near;
Table: Near 
oid1    oid2    vid    fid
0   1   1936   15
0   1   1936   16
0   1   1936   17
0   1   1936   18
0   1   1936   19
0   1   1936   20
0   1   1936   21
0   1   1936   22
0   1   1936   23
0   1   1936   24
0   1   2492   0
0   1   2492   1
0   1   2492   2
0   1   2492   3
0   1   2492   4
0   1   4135   11
0   1   4135   12
0   1   4135   13
0   1   4135   14
0   1   4417   17
0   1   4417   18
0   1   4417   19
0   1   4417   20
0   1   4417   21
0   1   4417   22
0   1   4417   23
0   1   4752   4
0   1   5818   3
0   1   5818   4
0   1   6947   31
*/ 
    
/*
All rows from table Far:    
SELECT * FROM Far;
Table: Far
oid1    oid2    vid    fid
0   1   595   0
0   1   595   1
0   1   595   2
0   1   595   3
0   1   595   4
0   1   595   5
0   1   595   6
0   1   595   7
0   1   595   31
0   1   836   0
0   1   836   1
0   1   836   2
0   1   836   3
0   1   836   4
0   1   836   5
0   1   836   6
0   1   836   7
0   1   836   8
0   1   836   9
0   1   836   10
0   1   836   11
0   1   836   12
0   1   836   13
0   1   836   14
0   1   836   15
0   1   836   16
0   1   836   17
0   1   836   18
0   1   836   19
0   1   836   20
0   1   836   21
0   1   836   22
0   1   836   23
0   1   836   24
0   1   836   25
0   1   836   26
0   1   836   27
0   1   1474   0
0   1   1474   1
0   1   1474   2
0   1   1474   3
0   1   1474   4
0   1   1474   5
0   1   1474   6
0   1   1474   7
0   1   1474   8
0   1   1474   9
0   1   1474   10
0   1   1474   11
0   1   1474   12
0   1   1474   13
0   1   1474   14
0   1   1474   15
0   1   1474   16
0   1   1474   17
0   1   1474   18
0   1   1474   19
0   1   1474   20
0   1   1474   21
0   1   1474   22
0   1   1474   23
0   1   1474   24
0   1   1474   25
0   1   1474   26
0   1   1474   27
0   1   2492   18
0   1   2492   19
0   1   2492   20
0   1   2492   21
0   1   2492   22
0   1   2492   23
0   1   2492   24
0   1   2492   25
0   1   2492   26
0   1   2492   27
0   1   2492   28
0   1   2492   29
0   1   2492   30
0   1   2492   31
0   1   2788   0
0   1   2788   1
0   1   2788   2
0   1   2788   25
0   1   2788   26
0   1   2788   27
0   1   2788   28
0   1   2788   29
0   1   2788   30
0   1   2788   31
0   1   2919   0
0   1   2919   1
0   1   2919   2
0   1   2919   3
0   1   2919   4
0   1   2919   5
0   1   2919   6
0   1   2919   7
0   1   2919   8
0   1   2919   9
0   1   2919   10
0   1   2919   11
0   1   2919   12
0   1   2919   13
0   1   2919   14
0   1   2919   15
0   1   2919   16
0   1   2919   17
0   1   2919   18
0   1   2919   19
0   1   2919   20
0   1   2919   21
0   1   2919   22
0   1   2919   23
0   1   2919   24
0   1   2919   25
0   1   2919   26
0   1   2919   27
0   1   2919   28
0   1   2919   29
0   1   2919   30
0   1   2919   31
0   1   4135   0
0   1   4135   1
0   1   4135   2
0   1   4135   3
0   1   4135   25
0   1   4135   26
0   1   4135   27
0   1   4135   28
0   1   4417   0
0   1   4417   1
0   1   4417   2
0   1   4417   3
0   1   4417   4
0   1   4417   5
0   1   4417   6
0   1   4417   7
0   1   4417   8
0   1   4417   9
0   1   4417   10
0   1   4417   11
0   1   4752   0
0   1   4752   1
0   1   4752   9
0   1   4752   13
0   1   4752   14
0   1   4752   15
0   1   4752   16
0   1   4752   17
0   1   4860   0
0   1   4860   1
0   1   4860   2
0   1   4860   3
0   1   4860   4
0   1   4860   5
0   1   4860   6
0   1   4860   7
0   1   4860   8
0   1   4860   9
0   1   5363   0
0   1   5363   1
0   1   5363   2
0   1   5363   3
0   1   5363   4
0   1   5363   5
0   1   5363   6
0   1   5363   7
0   1   5363   8
0   1   5363   9
0   1   5363   10
0   1   5363   11
0   1   5363   12
0   1   5363   13
0   1   5363   14
0   1   5363   15
0   1   5363   16
0   1   5363   17
0   1   5363   18
0   1   5363   19
0   1   6785   0
0   1   6785   1
0   1   6785   22
0   1   6785   23
0   1   6785   24
0   1   6785   25
0   1   6785   26
0   1   6785   27
0   1   6785   28
0   1   6947   0
0   1   6947   1
0   1   6947   2
0   1   6947   3
0   1   6947   4
0   1   6947   5
0   1   6947   6
0   1   6947   7
0   1   6947   8
0   1   6947   9
0   1   6947   10
0   1   6947   11
0   1   6947   12
0   1   7738   0
0   1   7738   1
0   1   7738   2
0   1   7738   3
0   1   7738   4
0   1   7738   5
0   1   7738   6
0   1   7738   7
0   1   7738   8
0   1   7738   9
0   1   7738   10
0   1   7738   11
0   1   7738   12
0   1   7738   13
0   1   7738   14
0   1   7738   15
0   1   7738   16
0   1   7738   17
0   1   7738   18
0   1   7738   19
0   1   7738   20
0   1   7738   21
0   1   7738   22
0   1   7738   23
0   1   7738   24
0   1   7738   25
0   1   7738   26
0   1   7738   27
0   1   7738   28
0   1   7738   29
0   1   7738   30
0   1   7738   31
0   1   9575   0
0   1   9575   1
0   1   9887   0
0   1   9887   1
0   1   9887   2
0   1   9887   3
0   1   9887   4
0   1   9887   5
0   1   9887   6
0   1   9887   7
0   1   9887   8
0   1   9887   9
0   1   9887   10
0   1   9887   11
*/

--- Using valid Postgres SQL, but don't use subqueries or literals, 
--- generate a query that, when executed on the tables provided above, returns the following table Output:

/*
All rows from table Output:    
SELECT * FROM Output;
Table: Output
vid
2492 
4752
4135
*/
    
SELECT
  """,
  temperature=0,
  max_tokens=500,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["#", ";"]
)



output = response.choices[0].text.replace('\n', ' ')
output = ' '.join(output.split())
output 

'vid FROM Near WHERE fid IN ( SELECT fid FROM Far WHERE oid1 = 0 AND oid2 = 1 ) GROUP BY vid HAVING COUNT(*) = ( SELECT COUNT(*) FROM Far WHERE oid1 = 0 AND oid2 = 1 ) ORDER BY vid'

In [13]:
gt_query = "Near_1(o0, o1); Far_3(o0, o1)"
predicate_list = ("Near_1", "Far_3")
synthesized_query = 'SELECT vid FROM Near WHERE fid IN ( SELECT fid FROM Far WHERE oid1 = 0 AND oid2 = 1 ) GROUP BY vid HAVING COUNT(*) = ( SELECT COUNT(*) FROM Far WHERE oid1 = 0 AND oid2 = 1 ) ORDER BY vid'

evaluate_patsql(gt_query, predicate_list, 5432, 4, synthesized_query)

score: 0.0, runtime: 0.058830976486206055
